In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from io import StringIO
from scipy.io import mmread

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls

'11_24_23 - Replicating Experiment.ipynb'                  eq2.ipynb
'11_25_23 - Second Degree Neighbors.ipynb'                 EQ3.ipynb
 11_26_23_Second_Degree_Model_2_Data.csv                   ia-email-univ.mtx
'11_26_23 - Test Statistics.ipynb'                         ia-infect-dublin.mtx
'12_5_23 - Simulate Data on Precreated Structure.ipynb'   'simulated data'/
 bio-grid-mouse.edges                                     'stats 209 project proposal.gdoc'
'[don'\''t use] 11_25_23_Second_Degree_Model_1_Data.csv'   Untitled0.ipynb


In [ ]:
edges_df = pd.read_csv('bio-grid-mouse.edges', header=None, delimiter=' ', names=['Node1', 'Node2'])

# Create a graph from the edges DataFrame
G = nx.from_pandas_edgelist(edges_df, 'Node1', 'Node2')

# Create an adjacency matrix
A = nx.adjacency_matrix(G).todense()

# Display the adjacency matrix
print("Adjacency Matrix:")
print(A)


Adjacency Matrix:
[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]


In [ ]:
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000
vertices = len(A[0])

In [ ]:
replicate = {}

In [ ]:
for pi in pi_vals:
  ADE = []
  AIE = [] # poonam said sus abt this?
  for i in range(1):
    #G = nx.random_regular_graph(adjacencies, vertices)
    W = np.random.binomial(1, pi, vertices)
    noise = np.random.normal(0, 1, vertices)
    Y = (W @ A)/(2* np.sum(A, axis=1)) + W * (1/2) + noise
    average_direct_effect = (np.dot(Y,W))/sum(W) - (np.dot(Y, (1-W)))/sum((1-W))
    ADE.append(average_direct_effect)
    average_indirect_effect = ((np.dot(A @ Y, W))/sum(W) - (np.dot(A @ Y, (1-W)))/sum(1-W))/adjacencies
    AIE.append(average_indirect_effect)

  str1 = str(pi)+" ADE"
  str2 = str(pi)+" AIE"
  replicate[str1] = ADE
  replicate[str2] = AIE


In [ ]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

,0.25 ADE,0.25 AIE,0.5 ADE,0.5 AIE,0.75 ADE,0.75 AIE
0,0.292593,0.007865,0.520693,0.005347,0.484559,-0.002742
1,0.436205,0.007397,0.465976,-0.000941,0.566717,0.008701
2,0.611712,0.001516,0.505458,-0.004370,0.530351,0.010775
3,0.324083,0.005792,0.437894,0.003998,0.540913,-0.001015
4,0.433082,-0.001668,0.549263,0.011135,0.618826,0.002075


In [ ]:
df_r.to_csv('12_5_23_Simulate_Model_1_1st_0505_Data_GRIDMOUSE.csv')

In [ ]:
df_r.mean()

0.25 ADE    0.667566
0.25 AIE    0.002179
0.5 ADE     0.668474
0.5 AIE     0.002125
0.75 ADE    0.665529
0.75 AIE    0.002594
dtype: float64

In [ ]:
df_r.std()

0.25 ADE    0.076446
0.25 AIE    0.007565
0.5 ADE     0.063931
0.5 AIE     0.006480
0.75 ADE    0.072309
0.75 AIE    0.007174
dtype: float64

In [ ]:
results = {'pi': [0.25, 0.5, 0.75], 'ADE': [0.667566, 0.668474, 0.665529],
'AIE': [0.002179, 0.002125, 0.002594]}

In [ ]:
df_r2 = pd.DataFrame.from_dict(results)
df_r2['INF'] = df_r2['ADE'] + df_r2['AIE']


In [ ]:
pd.options.plotting.backend = "plotly"
df_r2.plot(x='pi', y=['ADE', 'AIE', 'INF'])